### Week-4 practice solutions. SQL CookBook (Chapter 5-7)

##### Connect to MySQL database (Sakila)

In [ ]:
config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'sakila',
  'raise_on_warnings': True
}

In [ ]:
!pip install mysql-connector-python

In [ ]:
# create a MySQL server connection object
import mysql.connector
mydb = mysql.connector.connect(**config)

In [ ]:
# cursor object
my_cursor = mydb.cursor(dictionary=True)

##### Chapter-5

information_schema.tables->storing information about all the tables in that particular schema

information_schema.column->storing information about all the columns in that particular table

information_schema.key_column_usage-> storing information about all the keys and indexes in that particular schema

information_schema.table_constraints -> storing information about all the constraints in all the tables in that particular schema

In [ ]:
my_cursor.execute(" SELECT table_name FROM information_schema.tables WHERE table_schema ='sakila' ")
my_result = my_cursor.fetchall()

In [ ]:
for table in my_result:
    print(table)

In [ ]:
# Just fetching the tables
print(f"Table: {my_result[0]['table_name']}")
print(f"Total number of tables: {len(my_result)}")

In [ ]:
for table_num in range(len(my_result)):
    table_name = my_result[table_num]['table_name']
    my_cursor.execute(f" SELECT column_name, data_type, ordinal_position FROM information_schema.columns WHERE table_schema ='sakila' AND table_name='{table_name}' ")
    column_names = my_cursor.fetchall()
    print()
    print(f'column names for table: {table_name}')
    print('-------------------------------------')
    print(column_names)

5.3 Listing Indexed Columns for a Table

In [ ]:
my_cursor.execute("show index from actor;")
my_index= my_cursor.fetchall()
for idx in my_index:
    print(idx)


5.4 Listing Constraints on a Table

In [ ]:
sql_query='''SELECT a.table_name,
       a.constraint_name,
       b.column_name,
       a.constraint_type
FROM information_schema.table_constraints AS a,
     information_schema.key_column_usage AS b
WHERE a.table_name = 'address'
AND a.table_schema = 'sakila'
and a.table_name = b.table_name
AND a.table_schema = b.table_schema
AND a.constraint_name = b.constraint_name;
'''

my_cursor.execute(sql_query)
constraints=my_cursor.fetchall()
for row in constraints:
    print(row)

5.5 Listing Foreign Keys Without Corresponding Indexes

In [ ]:
sql_query='''SELECT * FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE 
WHERE TABLE_SCHEMA = 'sakila' AND TABLE_NAME = 'address';
'''

my_cursor.execute(sql_query)
foreign_keys=my_cursor.fetchall()
for row in foreign_keys:
    print(row)

In [ ]:
sql_query='''SHOW INDEX FROM sakila.address;
'''

my_cursor.execute(sql_query)
foreign_index=my_cursor.fetchall()
for row in foreign_index:
    print(row)

Here, we can see that there is an index present on the foreign key on the column city_id. In MySQL, when you define a foreign key constraint, an index on the columns involved is automatically created if it does not exist. 


5.6 Using SQL to Generate SQL 

In [ ]:
sql_query='''SELECT CONCAT('SELECT COUNT(*) FROM ', table_schema, '.', table_name, ';') 
FROM information_schema.tables 
WHERE table_schema = 'sakila';
'''

my_cursor.execute(sql_query)
cnt=my_cursor.fetchall()
for row in cnt:
    print(list(row.values())[0])

In [ ]:
sql_query='''SELECT CONCAT('ALTER TABLE ', table_name,' DROP FOREIGN KEY ', constraint_name, ';') 
FROM information_schema.KEY_COLUMN_USAGE 
WHERE table_schema = 'sakila' AND REFERENCED_TABLE_NAME IS NOT NULL;
'''

my_cursor.execute(sql_query)
cnt=my_cursor.fetchall()
for row in cnt:
    print(list(row.values())[0])

You check the foreign key constraints are present in all the tables in the database

In [ ]:
# OPTIONAL
sql_query='''SELECT 
  table_name,
  column_name,
  constraint_name,
  referenced_table_name,
  referenced_column_name
FROM
  information_schema.KEY_COLUMN_USAGE
WHERE
  REFERENCED_TABLE_SCHEMA = 'sakila'
  AND REFERENCED_TABLE_NAME IS NOT NULL;
'''


my_cursor.execute(sql_query)
foreign_keys=my_cursor.fetchall()
for row in foreign_keys:
    print(row)

In [ ]:
sql_query='''SELECT CONCAT('INSERT INTO actor(actor_id, first_name, last_name) VALUES(', actor_id, ', "', first_name, '", "', last_name, '");') 
FROM actor 
WHERE actor_id = 10;
'''

my_cursor.execute(sql_query)
cnt=my_cursor.fetchall()
for row in cnt:
    print(list(row.values())[0])

5.7 Describing the Data Dictionary Views 

In [ ]:
##Query to list data dictionary views and their purposes:
sql_query='''SELECT table_name, table_comment 
FROM information_schema.TABLES 
WHERE TABLE_SCHEMA = 'sakila';

'''


my_cursor.execute(sql_query)
views=my_cursor.fetchall()
for row in views:
    print(row)

In [ ]:
##Query to describe the columns in a given data dictionary view:
sql_query='''SELECT COLUMN_NAME, COLUMN_COMMENT 
FROM information_schema.COLUMNS 
WHERE TABLE_SCHEMA = 'sakila' AND TABLE_NAME = 'actor';


'''


my_cursor.execute(sql_query)
views=my_cursor.fetchall()
for row in views:
    print(row)

##### Chapter-6
##### Working with Strings

##### Counting the occurances of a Charecter in a string

- Counting the number of open brackets in a string.

Make sure to use a different connection and different cursor object

In [ ]:
shreya_config = {
  'user': 'root',
  'password': 'root',
  'host': '127.0.0.1',
  'port': 8889,
  'database': 'shreya',
  'raise_on_warnings': True
}

In [ ]:
shreya_db = mysql.connector.connect(**shreya_config)
# cursor object
new_cursor = shreya_db.cursor(dictionary=True)

In [ ]:
new_cursor.execute('''SELECT * FROM emp''')
for emp_table in new_cursor:
    print(emp_table)


In [ ]:
new_cursor.execute('''SELECT * FROM dept''')
for dept_table in new_cursor:
    print(dept_table)


##### 6.4 Removing unwanted characters from string

- For Example, removing char 'P' from the name

In [ ]:
new_cursor.execute('''select ename,
            replace(
            replace(
            replace(
            replace(
            replace(ename,'A',''),'E',''),'I',''),'O',''),'U','')
            as stripped1
            from emp;''')
for characters in new_cursor:
    print(characters)

Using the my_cursor object as we are connecting to sakila database to execute this query

In [ ]:
my_cursor.execute(''' SELECT first_name, REPLACE(first_name, 'P', '') as strp1 FROM actor LIMIT 1''')
for repc in my_cursor:
    print(repc)

6.5 Separating Numeric and Character Data - Skip as it uses Translate function

6.6 Determining Whether a String Is Alphanumeric

In [ ]:
new_cursor.execute('''create view V as
    select ename as data
    from emp
    where deptno=10
    union all
    select concat(ename,', $',sal,'.00') as data
    from emp
    where deptno=20
    union all
    select concat(ename,deptno) as data
    from emp
    where deptno=30;''')
for V_view in new_cursor:
    print(V_view)

In [ ]:
new_cursor.execute('''select * from V''')
for V_view in new_cursor:
    print(V_view)

In [ ]:
new_cursor.execute('''select data from V where data regexp '[^0-9a-zA-Z]' = 0;''')
for V_view in new_cursor:
    print(V_view)

6.7 Extracting Initials from a Name

Using the my_cursor object as we are connecting to sakila database for this query

In [ ]:
my_cursor.execute('''SELECT ((LENGTH('334 Munger (Monghyr) Lane')) - LENGTH(REPLACE('334 Munger (Monghyr) Lane', '(', ''))) / LENGTH("(") AS cnt ''')
for cnt in my_cursor:
    print(cnt)

In [ ]:
my_cursor.execute('''
    SELECT * from actor
''')
for cnt in my_cursor:
    print(cnt)

In [ ]:
my_cursor.execute('''
    SELECT first_name, last_name, CONCAT(SUBSTRING(SUBSTRING_INDEX(first_name, ' ', 1), 1, 1), '.', SUBSTRING(SUBSTRING_INDEX(last_name, ' ', -1), 1, 1)) AS initials
    FROM actor;
''')
for cnt in my_cursor:
    print(cnt)

6.8 Ordering by Parts of a String

In [ ]:
## ordering by last two characters
new_cursor.execute('''select ename from emp order by substr(ename,length(ename)-1,2);''')
for sub in new_cursor:
    print(sub)

In [ ]:
my_cursor.execute(''' SELECT title FROM film ORDER BY SUBSTR(title, LENGTH(title) - 1) LIMIT 5; ''')
conc_res = my_cursor.fetchall()
print(conc_res)

6.9 Ordering by a Number in a String - Cannot execute as MySQL does not provide the TRANSLATE function

6.10 Creating a Delimited List from Table Rows

In [ ]:
new_cursor.execute('''select deptno, group_concat(ename order by empno separator ',') as emps from emp group by deptno;''')
for delimit in new_cursor:
    print(delimit)

6.11 Converting Delimited Data into a Multivalued IN-List

In [ ]:
## here, the IN LIST ( '7654,7698,7782,7788' ) is considered to be a string rather than a comma delimited list
new_cursor.execute('''select empno, ename,sal,deptno
    from emp
    where empno in ( 7654,7698,7782,7788 );''')
for delimit in new_cursor:
    print(delimit)

In [ ]:
new_cursor.execute('''select ename,sal,deptno
    from emp
    where empno= 7654;''')
for delimit in new_cursor:
    print(delimit)

In [ ]:
new_cursor.execute('''CREATE TEMPORARY TABLE temp_numbers (id INT);''')
for temp in new_cursor:
    print(temp)

In [ ]:
new_cursor.execute('''INSERT INTO temp_numbers (id) VALUES 
(1), (2), (3), (4), (5), (6), (7), (8), (9), (10),
(11), (12), (13), (14), (15), (16), (17), (18), (19), (20);''')
for temp in new_cursor:
    print(temp)

In [ ]:
new_cursor.execute('''select empno, ename, sal, deptno 
    from emp
    where empno in
    ( select substring_index(
     substring_index(list.vals,',',iter.id),',',-1) empno
    from temp_numbers as iter,
    (select '7654,7698,7782,7788' as vals
    from (select 1) as dummy_table
    )list
    where iter.id <=
    (length(list.vals)-length(replace(list.vals,',','')))+1
    );''')
for delimit in new_cursor:
    print(delimit)

6.12 Alphabetizing a String

In [ ]:
new_cursor.execute('''select ename, group_concat(c order by c separator '') AS sorted_ename
    from (
    select ename, substr(a.ename,iter.id,1) c
    from emp a, temp_numbers iter
    WHERE iter.id <= LENGTH(a.ename)
    ) x
    GROUP BY ename;''')
for alpha in new_cursor:
    print(alpha)

6.13 Identifying Strings That Can Be Treated as Numbers


In [ ]:
new_cursor.execute('''create view P as
    select concat(
    substr(ename,1,2),
    replace(cast(deptno as char(4)),' ',''),
    substr(ename,3,2)
    ) as mixed
    from emp
    where deptno = 10
    union all
    select replace(cast(empno as char(4)), ' ', '')
    from emp where deptno = 20
    union all
    select ename from emp where deptno = 30;''')
for View_P in new_cursor:
    print(View_P)

In [ ]:
new_cursor.execute('''select * from P''')
for View_P in new_cursor:
    print(View_P)

In [ ]:
new_cursor.execute('''select cast(group_concat(c order by pos separator '') as unsigned)
        as MIXED1
        from (
        select P.mixed, iter.id AS pos, substr(P.mixed,iter.id,1) as c from P,
        temp_numbers as iter where iter.id <= length(P.mixed)
        and ascii(substr(P.mixed,iter.id,1)) between 48 and 57)y
        
        group by mixed
        order by 1;''')
for str in new_cursor:
    print(str)

6.14 Extracting the nth Delimited Substring

In [ ]:
new_cursor.execute('''CREATE TABLE permanent_numbers (id INT PRIMARY KEY);''')
for str in new_cursor:
    print(str)

In [ ]:
new_cursor.execute('''INSERT INTO permanent_numbers VALUES (1), (2), (3),(4), (5), (6), (7), (8), (9), (10),
(11), (12), (13), (14), (15), (16), (17), (18), (19), (20);
''')
for temp in new_cursor:
    print(temp)

In [ ]:
new_cursor.execute('''CREATE VIEW C AS
    SELECT 'mo,larry,curly' AS name
    FROM permanent_numbers
    UNION ALL
    SELECT 'tina,gina,jaunita,regina,leena' AS name
    FROM permanent_numbers;''')
for View_C in new_cursor:
    print(View_C)

In [ ]:
new_cursor.execute('''select distinct  * from C;''')
for View_C in new_cursor:
    print(View_C)

In [ ]:
new_cursor.execute('''Select distinct name
        from (
        select iter.id AS pos,
        substring_index(
        substring_index(src.name,',',iter.id),',',-1) AS name
        from C src,
        temp_numbers iter
        where iter.id <=
        length(src.name)-length(replace(src.name,',',''))
        ) x
        where pos = 2 LIMIT 2;''')
for View_C in new_cursor:
    print(View_C)

In [ ]:
new_cursor.execute('''select iter.id AS pos, src.name
from temp_numbers iter, C src
where iter.id <= length(src.name) - length(replace(src.name, ',', '')) LIMIT 1;''')
for View_C in new_cursor:
    print(View_C)

In [ ]:
new_cursor.execute('''select iter.id AS pos, src.name AS name1,
    substring_index(src.name,',',iter.id) AS name2,
    substring_index(
    substring_index(src.name,',',iter.id),',',-1) AS name3
    from temp_numbers iter,
    C src
    where iter.id <= length(src.name) - length(replace(src.name,',','')) LIMIT 2;''')
for View_C in new_cursor:
    print(View_C)

6.15 Parsing an IP Address

In [ ]:
new_cursor.execute('''select substring_index(substring_index(y.ip,'.',1),'.',-1) a,
       substring_index(substring_index(y.ip,'.',2),'.',-1) b,
       substring_index(substring_index(y.ip,'.',3),'.',-1) c,
       substring_index(substring_index(y.ip,'.',4),'.',-1) d
from (select '92.111.0.2' as ip from temp_numbers LIMIT 1) y;
''')
for address in new_cursor:
    print(address)

6.16 Comparing Strings by Sound

In [ ]:
new_cursor.execute('''CREATE TABLE author_names (
    a_name VARCHAR(50));''')
for author in new_cursor:
    print(author)

In [ ]:
new_cursor.execute('''INSERT INTO author_names (a_name) VALUES
('Johnson'),
('Jonson'),
('Jonsen'),
('Jensen'),
('Johnsen'),
('Shakespeare'),
('Shakspear'),
('Shaekspir'),
('Shakespar');
''')

for author in new_cursor:
    print(author)

In [ ]:
new_cursor.execute('''select distinct an1.a_name as name1, an2.a_name as name2,
    SOUNDEX(an1.a_name) as Soundex_Name
    from author_names an1
    join author_names an2
    on (SOUNDEX(an1.a_name)=SOUNDEX(an2.a_name)
    and an1.a_name!=  an2.a_name);''')

for author in new_cursor:
    print(author)

6.17 Finding Text Not Matching a Pattern:  Skip as this is a specific Oracle Database query.

##### Chapter-7
- 7.1 Computing an Average

In [ ]:
# cursor object for sakila database
mydb = mysql.connector.connect(**config)
my_cursor = mydb.cursor(dictionary=True)

In [ ]:
my_cursor.execute('''SELECT AVG(amount) AS avg_amount FROM payment''')
avg_res = my_cursor.fetchall()
print(avg_res)

In [ ]:
my_cursor.execute('''SELECT AVG(amount) AS avg_amount FROM payment GROUP BY payment_date LIMIT 5''')
avg_res = my_cursor.fetchall()
print(avg_res)

In [ ]:
my_cursor.execute(''' SELECT MAX(amount) AS MAX_PAYMENT FROM payment ''')
minmax_res = my_cursor.fetchall()
print(minmax_res)

In [ ]:
my_cursor.execute(''' SELECT SUM(amount) AS TOTAL_PAYMENT FROM payment ''')
minmax_res = my_cursor.fetchall()
print(minmax_res)

In [ ]:
my_cursor.execute(''' SELECT COUNT(*) AS total FROM film ''')
total_rows = my_cursor.fetchall()
print(total_rows)

In [ ]:
# This will exclude all the Max and Min values (if the column contains more than one max and min)
my_cursor.execute(''' SELECT AVG(amount) AS avg_without_high_low_values FROM payment WHERE amount NOT IN ((SELECT MAX(amount) FROM payment), (SELECT MIN(amount) FROM payment)) ''')
avg_w = my_cursor.fetchall()
print(avg_w)

In [ ]:
# Use this to exclude only one set of Max and Min
my_cursor.execute(''' SELECT (SUM(amount) - MIN(amount) - MAX(amount)) / (COUNT(*) - 2) AS avg_wmm FROM payment''')
avg_without_max_min = my_cursor.fetchall()
print(avg_without_max_min)

7.6 Generating a Running: Run the queries from 7.6 in sqlite as these queries use window functions which is not supported by MySQL Workbench

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('week1.db')
c = conn.cursor()

In [ ]:
read_emp = pd.read_csv(r'/Users/shreyagajbhiye/Desktop/adt/adt_ta/week1/emp.csv')
read_emp.to_sql('emp', conn, if_exists='replace', index = False) # Insert the values from the csv file into the table 'emp'

In [ ]:
read_dept = pd.read_csv(r'/Users/shreyagajbhiye/Desktop/adt/adt_ta/week1/dept.csv')
read_dept.to_sql('dept', conn, if_exists='replace', index = False) # Insert the values from the csv file into the table 'dept'

7.6 Generating a Running Total

In [ ]:
c.execute('''
select ename, sal,
sum(sal) over (order by sal,empno) as running_total
from emp
order by 2;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select empno, sal,
sum(sal)over(order by sal,empno) as running_total1,
sum(sal)over(order by sal) as running_total2
from emp
order by 2;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

7.7 Generating a Running Product

In [ ]:
#if you are facing issues in exp and ln function, it is because of the version of MySQL that you are using.
import math

In [ ]:
conn.create_function("exp", 1, math.exp)
conn.create_function("ln", 1, math.log)
c= conn.cursor()

In [ ]:
c.execute('''
select empno,ename,sal,
exp(sum(ln(sal))over(order by sal,empno)) as running_prod
from emp
where deptno = 10;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

7.8 Smoothing a Series of Values - it is okay if you skip

7.9 Calculating a Mode

In [ ]:
c.execute('''
select sal
from emp
where deptno = 20
order by sal;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
# Mode is 3000
c.execute('''
    select sal
    from (
    select sal,
     dense_rank()over( order by cnt desc) as rnk
    from (
    select sal, count(*) as cnt
    from emp
    where deptno = 20
    group by sal
    ) x
    ) y
    where rnk = 1;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select sal,
dense_rank()over(order by cnt desc) as rnk
from (
select sal,count(*) as cnt
from emp
where deptno = 20
group by sal
) x;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

7.10 Calculating a Median


In [ ]:
c.execute('''
select sal
from emp
where deptno = 20
order by sal;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
with rank_tab (sal, rank_sal) as
(
select sal, cume_dist() over (order by sal)
from emp
where deptno=20
),
inter as
(
select sal, rank_sal from rank_tab
where rank_sal>=0.5
union
select sal, rank_sal from rank_tab
where rank_sal<=0.5
)
select avg(sal) as MedianSal
from inter;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

7.11 Determining the Percentage of a Total

In [ ]:
c.execute('''
select *
from emp where deptno=10;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select (CAST(sum(case when deptno = 10 then sal end) AS FLOAT)/CAST(sum(sal) AS FLOAT)
)*100 as pct
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select sum(case when deptno = 10 then sal end) as d10,
sum(sal)
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select (cast(
sum(case when deptno = 10 then sal end)
as FLOAT))/CAST(sum(sal) AS FLOAT
)*100 as pct
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select deptno,
sum(sal)over() total,
sum(sal)over(partition by deptno) d10
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select deptno,
sum(sal)over() total,
sum(sal)over(partition by deptno) d10
from emp
where deptno=10;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

7.12 Aggregating Nullable Columns

In [ ]:
c.execute('''
select avg(coalesce(comm,0)) as avg_comm
from emp
where deptno=30;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select avg(comm)
from emp
where deptno=30;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select ename, comm
from emp
where deptno=30
order by comm desc;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

7.13 Computing Averages Without High and Low Values

In [ ]:
c.execute('''
select avg(sal)
from emp
where sal not in (
(select min(sal) from emp),
(select max(sal) from emp)
);
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select (sum(sal)-min(sal)-max(sal))/(count(*)-2) AS cnt
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

In [ ]:
c.execute('''
select sal, min(sal)over() min_sal, max(sal)over() max_sal
from emp;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

Skip 7.14, 7.15 as it uses traslate function

7.16 Finding Outliers Using the Median Absolute
Deviation

In [ ]:
c.execute('''
    with rank_tab (sal, rank_sal) as (
    select sal, cume_dist() over (order by sal)
    from emp),
    inter as
    (
    select sal, rank_sal from rank_tab
    where rank_sal>=0.5
    union
    select sal, rank_sal from rank_tab
    where rank_sal<=0.5
    )
    ,
    medianSal (medianSal) as
    (
    select (max(sal)+min(sal))/2
    from inter),
    deviationSal (Sal,deviationSal) as
    (select Sal,abs(sal-medianSal)
    from emp join medianSal
    on 1=1
    )
    ,
    distDevSal (sal,deviationSal,distDeviationSal) as
    (
    select sal,deviationSal,cume_dist() over (order by deviationSal)
    from deviationSal
    ),
    DevInter (DevInter, sal) as
    (
    select min(deviationSal), sal
    from distDevSal
    where distDeviationSal >= 0.5
    union
    select max(DeviationSal), sal
    from distDevSal
    where distDeviationSal <= 0.5
    ),
    MAD (MedianAbsoluteDeviance) as
    (
    select abs(emp.sal-(min(devInter)+max(devInter))/2)
    from emp join DevInter on 1=1
    )
    select emp.sal,MedianAbsoluteDeviance,
    (emp.sal-deviationSal)/MedianAbsoluteDeviance as ratio
    from (emp join MAD on 1=1)
    join deviationSal on emp.sal=deviationSal.sal;
''')

colnames = c.description
colnames_list = []
for row in colnames:
    colnames_list.append(row[0])

df = pd.DataFrame(c.fetchall(), columns=colnames_list)
df

Skip 7.17 Finding Anomalies Using Benford’s Law

In [ ]:
conn.close()